In [1]:
import sys
sys.path.insert(0, '/notebooks/chatweb3')

In [2]:
import uuid
import os
import time
from typing import Any, Callable, Generator

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.base import BaseOutputParser

from chat.base import BaseChat, ChatHistory, Response, streaming_callback_manager
from chat.widget_search import ChatOutputParser, WIDGET_INSTRUCTION, SEARCH_INSTRUCTION, TEMPLATE, IDENTIFY_TEMPLATE
from chat.base import *
from index.weaviate import *
from index.widgets import *
from utils import *

Added to class registry: chat.simple.SimpleChat
Added to class registry: chat.rephrase.RephraseChat
Added to class registry: chat.rephrase_cited.RephraseCitedChat
Added to class registry: chat.widget_search.WidgetSearchChat
Added to class registry: index.weaviate.WeaviateIndex


In [3]:
set_api_key()

In [4]:
@registry.register_class
class WidgetSearchChat(BaseChat):
    def __init__(self, doc_index: Any, widget_index: Any, top_k: int = 3, show_thinking: bool = True) -> None:
        super().__init__()
        self.output_parser = ChatOutputParser()
        self.widget_prompt = PromptTemplate(
            input_variables=["task_info", "question"],
            template=TEMPLATE.replace("{instruction}", WIDGET_INSTRUCTION),
            output_parser=self.output_parser,
        )
        self.search_prompt = PromptTemplate(
            input_variables=["task_info", "question"],
            template=TEMPLATE.replace("{instruction}", SEARCH_INSTRUCTION),
            output_parser=self.output_parser,
        )
        self.doc_index = doc_index
        self.widget_index = widget_index
        self.top_k = top_k
        self.show_thinking = show_thinking

        self.identify_prompt = PromptTemplate(
            input_variables=["history", "question"],
            template=IDENTIFY_TEMPLATE,
            output_parser=self.output_parser,
        )

    def get_llm(self, new_token_handler):
        llm = OpenAI(
            temperature=0.0, max_tokens=-1,
            # options for streaming
            streaming=True, callback_manager=streaming_callback_manager(new_token_handler)
        )
        return llm

    def get_streaming_chain(self, prompt, new_token_handler):
        llm = self.get_llm(new_token_handler)
        chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
        return chain

    def receive_input(self, history: ChatHistory, userinput: str,) -> Generator[Response, None, None]:
        userinput = userinput.strip()
        # First identify the question
        history_string = ""
        for interaction in history:
            history_string += ("User: " + interaction.input + "\n")
            history_string += ("Assistant: " + interaction.response + "\n")
        start = time.time()
        example = {
            "history": history_string.strip(),
            "question": userinput,
            "stop": "##",
        }

        chat_message_id = None
        identify_response = ''
        identified_type = None
        sent_response = ''

        def identify_token_handler(token):
            nonlocal chat_message_id, identify_response, identified_type, sent_response
            if not self.show_thinking:
                return
            identify_response += token
            if '> ' not in identify_response.strip():
                return

            if not identified_type:
                identified_type, question = identify_response.strip().split(' ', 1)
                if identified_type == '<widget>':
                    token = "I think you want a widget for: " + question
                else:
                    token = "I think you're asking: " + question

            sent_response += token
            print('token = ', token)
            # chat_message_id = send(Response(
            #     response=token,
            #     still_thinking=False,
            #     actor='bot',
            #     operation='append' if chat_message_id is not None else 'create',
            # ), last_chat_message_id=chat_message_id)

        identify_chain = self.get_streaming_chain(self.identify_prompt, identify_token_handler)
        identify_response = identify_chain.apply_and_parse([example])[0]
        if self.show_thinking:
            # send again, but with replace so we save to db
            print('sent_response = ', sent_response)
            # send(Response(
            #     response=sent_response,
            #     still_thinking=False,
            #     actor='bot',
            #     operation='replace',  # this will save to db where append did not
            # ), last_chat_message_id=chat_message_id)

        duration = time.time() - start
        identified_type, question = identify_response.split(' ', 1)
        print(f'Intent identification took {duration: .2f}s')
        # send(Response(
        #     response=f'Intent identification took {duration: .2f}s',
        #     actor='system',
        #     still_thinking=True,  # turn on thinking again
        # ))

        chat_message_id = None

        def new_token_handler(token):
            nonlocal chat_message_id
            print('token = ', token)
            # chat_message_id = send(Response(
            #     response=token,
            #     still_thinking=False,
            #     actor='bot',
            #     operation='append' if chat_message_id is not None else 'create',
            # ), last_chat_message_id=chat_message_id)

        if identified_type == '<widget>':
            widgets = self.widget_index.similarity_search(question, k=self.top_k)
            task_info = '\n'.join([f'Widget: {widget.page_content}' for widget in widgets])
            chain = self.get_streaming_chain(self.widget_prompt, new_token_handler)
        else:
            docs = self.doc_index.similarity_search(question, k=self.top_k)
            task_info = '\n'.join([f'Content: {doc.page_content}\nSource: {doc.metadata["url"]}' for doc in docs])
            chain = self.get_streaming_chain(self.search_prompt, new_token_handler)
        start = time.time()
        example = {
            "task_info": task_info,
            "question": question,
            "stop": "User",
        }

        result = chain.apply_and_parse([example])[0]
        duration = time.time() - start
        history.add_interaction(userinput, result)
        print('result = ', result)
        print(f'Response generation took {duration: .2f}s')
        # send(Response(result, operation='replace'), last_chat_message_id=chat_message_id)
        # send(Response(response=f'Response generation took {duration: .2f}s', actor='system'))


Added to class registry: __main__.WidgetSearchChat


In [5]:
doc_index = WeaviateIndex('IndexV1', 'content') 
widget_index = WeaviateIndex('WidgetV1', 'content')

In [6]:
interactions, session_id = [], str(uuid.uuid4())
history = ChatHistory(interactions, session_id)

In [7]:
chat = WidgetSearchChat(doc_index, widget_index)

In [8]:
chat.receive_input(history, 'what are you capable of?')

Prompt after formatting:

Given the following conversation and a follow up response input, determine if the input is related to a command to invoke using a widget or if it is a search query for a knowledge base. If it is a widget, return the appropriate keywords to search for the widget, as well as all relevant details to invoke it. If it is a search query, rephrase as a standalone question. You should assume that the query is related to web3.

## Example:

Chat History:
User: I'd like to make transfer ETH
Assistant: Ok I can help you with that. How much and to which address?

Input: 2 ETH to andy
Ouput: <widget> transfer of 2 ETH currency to andy

## Example:

Chat History:
User: Who created Ethereum?
Assistant: Vitalik Buterin
User: What about AAVE?
Assistant: Stani Kulechov

Input: When was that?
Output: <query> When were Ethereum and AAVE created?

## Example:

Chat History:
User: Who created Ethereum?
Assistant: Vitalik Buterin

Input: What is AAVE?
Output: <query> What is AAVE?



KeyError: 'url'

In [9]:
docs = doc_index.similarity_search('what are your capabilities', k=3)

In [10]:
docs[1]

Document(page_content="molded by their previous experiences.\n\n5.3. Do-ers vs Talkers\n\nHire do-ers who can express themselves, not talkers who can’t do. Doers who can’t articulate are tricky. They may be ok in some narrow tech situations, but we can’t have too many of them on the team either.\n\n5.4. Hire with a Goal\n\nEach new person must have clear responsibilities, ideally with aggressive numerical targets with about a 70% chance of success.\n\n5.5. No Titles\n\nDon’t hire people who are worried about titles. It’s not a deal breaker, but definitely not a good sign.\n\n5.6. Mission over Money\n\nDon’t hire people who are overly fixated on salaries and comp. We should pay people fairly.\n\n5.7. When in doubt, don’t hire.\n\nIf you have doubts during the hiring process, don’t hire. Small doubts in the interview stage always turn into a big problem in the future.\n\n6. Leadership Style\n6.1. Don’t try to motivate people who are not self motivated.\xa0\n\nIt’s like dragging a dead ho